In [33]:
import numpy as np
from scipy.spatial import distance
import pandas as pd
from scipy import stats
import operator
from sklearn.model_selection import KFold
from statistics import mean

np.set_printoptions(suppress=True) #prevent numpy exponential

def read_data(file):
    nominal = dict()
    gene_data = open(file)
    gene_seq = gene_data.readlines()
    all_genes_list = []
    first_row = gene_seq[0].split("\t")
    for f in range(len(first_row)):
        try:
            float(first_row[f])
        except:
            nominal[f] = []
    for line in gene_seq:
        gene = line.strip().split("\t")
        for f in range(len(gene)):
            try:
                gene[f] = float(gene[f])
            except:
                category = nominal[f]
                if gene[f] in category:
                    gene[f] = float(category.index(gene[f]))
                else:
                    category.append(gene[f])
                    gene[f] = float(category.index(gene[f]))
        all_genes_list.append(gene)
    return nominal, np.asarray(all_genes_list, dtype = float)

def compute_naive_bayes(train_data, test_data, train_classes, nominal_dict):
    class_dict = dict()
    stat_dict = dict()
    predicted_classes = []
    for i in range(len(train_classes)):
        class_dict.setdefault(train_classes[i],[]).append(train_data[i])
    for k in class_dict:
        mean = np.mean(class_dict[k], axis = 0)
        std = np.std(class_dict[k], axis = 0)
        stat_dict[k] = (mean, std)
    
    for i in range(len(test_data)):
        test_record = test_data[i]
        max_prob = -1
        label = -1
        for k in class_dict:
            prob = 1.0
            m = stat_dict[k][0]
            sigma = stat_dict[k][1]
            for feature in range(len(test_record)):
                if feature in nominal_dict:
                    count = list(np.asarray(class_dict[k])[:, feature]).count(test_record[feature])
                    prob *= count/len(class_dict[k])
                else:
                    prob *= stats.norm(m[feature], sigma[feature]).pdf(test_record[feature])
            prob *= len(class_dict[k])/len(train_data)
            if prob > max_prob:
                max_prob = prob
                label = k
        predicted_classes.append(label)
    return predicted_classes         
def calculate_metrics(predicted_classes, ground_truth):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(0, len(predicted_classes)):
        if(predicted_classes[i] == 1 and ground_truth[i] == 1):
            tp += 1
        elif(predicted_classes[i] == 1 and ground_truth[i] == 0):
            fp += 1
        elif(predicted_classes[i] == 0 and ground_truth[i] == 1):
            fn += 1
        else:
            tn += 1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    if (tp+fp) != 0:
        precision = tp / (tp + fp)   
    if (tp+fn) != 0:
        recall = tp / (tp + fn)   
    if ((2 * tp) + fp + fn) != 0:
        f_1_measure = (2 * tp) / ((2 * tp) + fp + fn)    
    return accuracy, precision, recall, f_1_measure  

nominal_dict, data = read_data("project3_dataset2.txt")
classes = data[:,len(data[0])-1]
feature_data = data[:,:len(data[0])-1]
kfold = KFold(10, True, 1)
accuracy_list = []
precision_list = []
recall_list = []
f_1_measure_list = []
for train, test in kfold.split(feature_data):
    train_data = feature_data[train]
    test_data = feature_data[test]
    train_classes = classes[train]
    test_classes = classes[test]
    print(len(train_classes), len(test_classes))
    predicted_classes = compute_naive_bayes(train_data, test_data, train_classes, nominal_dict)
    accuracy, precision, recall, f_1_measure = calculate_metrics(predicted_classes, test_classes)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f_1_measure_list.append(f_1_measure)
print("Accuracy: ", mean(accuracy_list))
print("Precision: ", mean(precision_list))
print("Recall: ", mean(recall_list))
print("F_1_measure: ", mean(f_1_measure_list))

415 47
415 47
416 46
416 46
416 46
416 46
416 46
416 46
416 46
416 46
Accuracy:  0.703422756706753
Precision:  0.5656792717086835
Recall:  0.6165857658853015
F_1_measure:  0.5852148100687
